# Dataframe for Dynamic Pricing

## Objectives

This document intends to briefly describe one alternative for the ETL process for the Dynamic Pricing problem at Olist.  
As the end product, we would like to have a table with the proper structure for forecasting techniques.  
The datasets will be described along the way.

### Importing packages

In [1]:
import pandas as pd
import datetime

### Dataframe with complete range

Let's start by creating a dataframe with every data point we want to measure. It will contain hourly intervals, equally spaced between two dates.  
You can change the parameters in the cell below:

In [2]:
freq = 'D' # 'H' is for hourly data. You can use 'D' for days, 'W' for last day of week, 'Q' for quarter, etc.

start_date = '1/1/2017' # MM/DD/YYYY
end_date = '4/1/2020' # MM/DD/YYYY

In [3]:
df = pd.DataFrame(data=pd.date_range(start=start_date, end=end_date, freq=freq),
                  columns=['date'])

df.head(2)

,date
0,2017-01-01
1,2017-01-02


In [4]:
df.tail(2)

,date
1185,2020-03-31
1186,2020-04-01


### Orders dataset

Let's investigate our first dataset "orders.csv". It contains information for every sale that was executed in a given period of time.  
Our main idea is to aggregate sales data and merge it with our dates dataframe (df).

In [5]:
# READING ORDERS DATA AND TRANSFORMING COLUMNS
orders = pd.read_csv('data/orders.csv', dtype={'product_gtin':'str','origin_zip':'str',
                                               'destination_zip':'str'})

orders['purchase_timestamp'] = pd.to_datetime(orders['purchase_timestamp'])

In [6]:
orders.head()

,purchase_timestamp,seller_id,product_gtin,product_sku,channel_slug,channel_store,price,freight_value,price_freight_shift,commission_product_olist,commission_freight_olist,origin_zip,origin_state,origin_city,destination_zip,destination_state,destination_city
0,2019-07-20 11:19:43,a97001a2-a7d2-4bd6-b0d5-1da2a2b8abd9,6952202281890,PRDWRHUYLLMFX2U0,mercadolivre,mercadolivre,82.99,20.86,0.0,0.20,0.20,01109010,SP,SAO PAULO,89265575,SC,Jaraguá do Sul
1,2019-11-11 20:43:23,a97001a2-a7d2-4bd6-b0d5-1da2a2b8abd9,6951003643197,PRDACYVT895VMO7H,mercadolivre,mercadolivre,119.90,0.00,0.0,0.20,0.20,01109010,SP,SAO PAULO,06040470,SP,Osasco
2,2020-03-01 16:25:00,c58b83e5-52de-43fe-8399-a7bc312e2aa5,7898907175302,PRDKTDU50G9FVV57,cnova,ex,39.90,14.24,0.0,0.19,0.19,20910970,RJ,RIO DE JANEIRO,22260210,RJ,Rio de Janeiro
3,2020-02-04 12:04:10,a97001a2-a7d2-4bd6-b0d5-1da2a2b8abd9,6951003643197,PRDACYVT895VMO7H,mercadolivre,mercadolivre,130.95,0.00,0.0,0.20,0.20,01109010,SP,SAO PAULO,89062100,SC,Blumenau
4,2018-12-14 13:52:19,a97001a2-a7d2-4bd6-b0d5-1da2a2b8abd9,6951003643197,PRDACYVT895VMO7H,mercadolivre,mercadolivre,128.99,21.51,0.0,0.20,0.20,01109010,SP,SAO PAULO,66055000,PA,Belém


In [7]:
orders.product_gtin.nunique()

647

###### Selecting specific product

In the orders dataset, we have sales information for 647 different products.  
For simplification purposes, let's choose one single product for analysis and manipulation of the dataset.  
To do so, choose a single gtin (str) in the cell below.

- gtin = '7908243600137'
- product_sku = 'PRD41RHHBF9NQR74'
- canonical_sku = 'PRDIM5XOLWMUM0HU'


In [8]:
filter_orders = orders[['purchase_timestamp','product_gtin',
                        'price','freight_value',
                        'commission_product_olist','commission_freight_olist']].copy()

# ADD ARTIFICIAL COLUMN WITH THE NUMBER OF ORDERS -> WILL BE USED TO COUNT THE ORDERS BY HOUR
filter_orders['orders'] = 1


# GROUPS ORDERS BY CHOSEN FREQUENCY
filter_orders = filter_orders.groupby([pd.Grouper(key='purchase_timestamp', freq=freq),'product_gtin'])\
                  .agg({'price':'mean', 
                        'freight_value':'mean',
                        'commission_product_olist':'mean',
                        'commission_freight_olist':'mean',
                        'orders':'sum'}).reset_index()

In [9]:
filter_orders.head()

,purchase_timestamp,product_gtin,price,freight_value,commission_product_olist,commission_freight_olist,orders
0,2015-09-23,7899893323067,168.0,0.000,0.2,0.0,1
1,2015-10-13,7899893323067,168.0,0.000,0.2,0.0,1
2,2016-04-16,7898152189550,343.9,17.060,0.2,0.0,1
3,2016-06-10,7897889805825,129.9,28.540,0.2,0.0,3
4,2016-06-11,7897889805825,129.9,32.534,0.2,0.0,5


##### merge date_range dataframe and orders

Now that we have the number of sales of a given product by hour, along with its average price, average shipping values and average commissions (percentage), we can join with our date range dataframe.  

The 'NaN' rows are the ones that did not have any sales.

In [10]:
# MERGE DF and Filter orders

merged = pd.merge(df, filter_orders, how='left', left_on='date', right_on='purchase_timestamp')
merged.drop('purchase_timestamp', axis=1, inplace=True)

In [11]:
merged.head()

,date,product_gtin,price,freight_value,commission_product_olist,commission_freight_olist,orders
0,2017-01-01,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-01-02,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-01-03,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-01-04,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-01-05,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Checking some rows with sales

merged[merged['orders'].notnull()].head()

,date,product_gtin,price,freight_value,commission_product_olist,commission_freight_olist,orders
5,2017-01-06,7891530258930,579.99,28.55,0.2,0.0,1.0
7,2017-01-08,7898914878487,66.90,18.75,0.2,0.0,1.0
8,2017-01-09,7898914878487,66.90,18.75,0.2,0.0,2.0
10,2017-01-11,7891530258930,579.99,18.23,0.2,0.0,1.0
11,2017-01-11,7897889805825,129.99,29.19,0.2,0.0,1.0


### Product_info dataset

Another available dataset is the "product_info.csv". We will use it to check when a product was created in our database to fine tune the starting point of our time series.

In [13]:
prod_info = pd.read_csv('data/product_info.csv', dtype={'gtin':'str'})

prod_info['created_at'] = pd.to_datetime(prod_info['created_at'])

In [14]:
filter_prod = prod_info[['created_at','gtin','name']]

In [15]:
filter_prod = filter_prod.groupby(['gtin','name']).min().reset_index()

In [17]:
def truncate_date(x):
    return datetime.date(x.year,x.month,x.day)

In [18]:
filter_prod['created_date'] = filter_prod['created_at'].map(truncate_date)

In [19]:
filter_prod.head()

,gtin,name,created_at,created_date
0,1001001118649,"Esqueleto 170 cm, Padrão, com Suporte e Base c...",2019-06-03 14:50:41,2019-06-03
1,1003001113372,Autoclave Vertical 18 Litros,2019-06-21 13:05:50,2019-06-21
2,1003001209488,"Mini Centrífuga Para 8 Microtubos Vol. 0,2 A 2...",2019-12-02 12:38:41,2019-12-02
3,1234567896618,"Veda Fresta De Portão 6,4 Cm Altura Vedapratic...",2019-07-01 09:24:29,2019-07-01
4,1286508320126,Diabete composto 90 Cápsulas 500 Mg insulina n...,2019-01-24 13:10:00,2019-01-24


In [32]:
prod_info[prod_info['gtin']=='1001001118649']

,gtin,part_number,canonical_sku,seller_product_sku,name,description,external_id,channel_slug,price,offer,stock,created_at,updated_at
750,1001001118649,NaN,PRDYBAH5LASBS226,PRD8CK2BKF9LDAJ6,"Esqueleto 170 cm, Padrão, com Suporte e Base c...","Modelo anatômico de esqueleto humano, medindo ...",47565971,cnova,1260.0,1260.0,9,2019-06-03 14:50:42,2020-02-12 21:44:30
800,1001001118649,NaN,PRD5AOADTTLG8ZDH,PRD8CK2BKF9LDAJ6,"Esqueleto 170 cm, Padrão, com Suporte e Base c...","Modelo anatômico de esqueleto humano, medindo ...",NaN,zoom,1280.0,1280.0,5,2019-06-03 14:50:45,2019-06-03 14:51:14
801,1001001118649,NaN,PRD7MNGYFIPKK2CV,PRD8CK2BKF9LDAJ6,"Esqueleto 170 cm, Padrão, com Suporte e Base c...","Modelo anatômico de esqueleto humano, medindo ...",NaN,amazon,1280.0,1280.0,5,2019-06-03 14:50:41,2019-08-03 19:07:25
2704,1001001118649,NaN,PRDE1HXHV5LHSGKK,PRD8CK2BKF9LDAJ6,"Esqueleto 170 cm, Padrão, com Suporte e Base c...","Modelo anatômico de esqueleto humano, medindo ...",MLB1243740907,mercadolivre,1260.0,1260.0,9,2019-06-03 14:50:44,2020-02-12 21:44:25
3016,1001001118649,NaN,PRDVITLYWBJZKIY1,PRD8CK2BKF9LDAJ6,"Esqueleto 170 cm, Padrão, com Suporte e Base c...","Modelo anatômico de esqueleto humano, medindo ...",NaN,saraiva,1280.0,1280.0,5,2019-06-03 14:50:44,2019-06-03 14:51:02
3098,1001001118649,NaN,PRDVKSOE0WA8PWAB,PRD8CK2BKF9LDAJ6,"Esqueleto 170 cm, Padrão, com Suporte e Base c...","Modelo anatômico de esqueleto humano, medindo ...",NaN,luanet,1280.0,1280.0,5,2019-06-03 14:50:43,2019-06-03 14:51:02
4472,1001001118649,NaN,PRD9HKY68HWMFU12,PRD8CK2BKF9LDAJ6,"Esqueleto 170 cm, Padrão, com Suporte e Base c...","Modelo anatômico de esqueleto humano, medindo ...",81955387,b2w,1260.0,1260.0,9,2019-06-03 14:50:41,2020-02-12 22:30:36
4481,1001001118649,NaN,PRDK9DSPEV6SOQWL,PRD8CK2BKF9LDAJ6,"Esqueleto 170 cm, Padrão, com Suporte e Base c...","Modelo anatômico de esqueleto humano, medindo ...",NaN,madeiramadeira,1280.0,1280.0,5,2019-06-03 14:50:43,2019-06-03 14:51:14
4820,1001001118649,NaN,PRD6W7N80XSZ7YOE,PRD8CK2BKF9LDAJ6,"Esqueleto 170 cm, Padrão, com Suporte e Base c...","Modelo anatômico de esqueleto humano, medindo ...",NaN,carrefour,1280.0,1280.0,5,2019-06-03 14:50:41,2019-12-23 17:07:51
6024,1001001118649,NaN,PRDAVFBEN7OD0SST,PRD8CK2BKF9LDAJ6,"Esqueleto 170 cm, Padrão, com Suporte e Base c...","Modelo anatômico de esqueleto humano, medindo ...",NaN,digitalweb,1280.0,1280.0,5,2019-06-03 14:50:42,2019-06-03 14:51:49


##### Filter merged based on creation date

In [20]:
merged = pd.merge(merged, filter_prod, left_on='product_gtin', right_on='gtin', how='left')

In [21]:
merged['created_date'] = pd.to_datetime(merged['created_date'])

In [22]:
# Keep dates only where the product existed.
merged = merged[merged['date']>=merged['created_date']].copy()

In [23]:
merged.head()

,date,product_gtin,price,freight_value,commission_product_olist,commission_freight_olist,orders,gtin,name,created_at,created_date
5,2017-01-06,7891530258930,579.99,28.55,0.2,0.0,1.0,7891530258930,Relógio Technos Ts Carbon Os10er/1r.,2016-07-24 13:30:02,2016-07-24
7,2017-01-08,7898914878487,66.90,18.75,0.2,0.0,1.0,7898914878487,Caixa de Pesca Com 6 Bandejas Articuladas CX-6...,2016-10-02 22:16:36,2016-10-02
8,2017-01-08,7898914878487,66.90,18.75,0.2,0.0,1.0,7898914878487,Caixa de pesca 6 bandejas com divisórias model...,2016-10-17 15:23:04,2016-10-17
10,2017-01-09,7898914878487,66.90,18.75,0.2,0.0,2.0,7898914878487,Caixa de Pesca Com 6 Bandejas Articuladas CX-6...,2016-10-02 22:16:36,2016-10-02
11,2017-01-09,7898914878487,66.90,18.75,0.2,0.0,2.0,7898914878487,Caixa de pesca 6 bandejas com divisórias model...,2016-10-17 15:23:04,2016-10-17


### Competition

Now, let's get our competitor's prices by the same frequency.

In [24]:
# Read data and format it
competition = pd.read_csv('data/competition.csv', dtype={'gtin':'str'})

competition['updated_at'] = pd.to_datetime(competition['updated_at'])

In [25]:
competition.head()

,updated_at,gtin,value,slug
0,2020-01-29 10:44:18,1001001118649,1260.0,sieve-b2w-submarino
1,2020-01-29 10:43:43,1001001118649,1260.0,sieve-b2w-shoptime
2,2020-01-29 10:42:41,1001001118649,1159.2,sieve-b2w-americanas
3,2020-01-24 15:17:21,1001001118649,0.0,sieve-b2w-shoptime
4,2020-01-24 15:17:20,1001001118649,0.0,sieve-b2w-americanas


In [35]:
# filter out zero values
competition = competition[competition['value'] > 0]

In [43]:
gr = competition.groupby('gtin').agg({'value':['mean', 'std']}).reset_index()

In [44]:
gr.head()

gtin        value           
                         mean        std
0  1001001118649  1256.575000  31.937300
1  1003001113372  5088.323333   4.106978
2  1003001209488  1011.585000  81.363378
3  1234567896618   103.908621   5.497400
4  1286508320126    26.643370   1.758233

In [47]:
comp_m = pd.merge(competition, gr, left_on='gtin', right_on='gtin')

In [50]:
comp_m.columns = ['updated_at', 'gtin', 'value', 'slug', 'mean','std']

In [53]:
comp_m['upper'] = comp_m['mean'].values + 3*comp_m['std'].values
comp_m['lower'] = comp_m['mean'].values - 3*comp_m['std'].values

In [54]:
comp_m.head()

,updated_at,gtin,value,slug,mean,std,upper,lower
0,2020-01-29 10:44:18,1001001118649,1260.0,sieve-b2w-submarino,1256.575,31.9373,1352.3869,1160.7631
1,2020-01-29 10:43:43,1001001118649,1260.0,sieve-b2w-shoptime,1256.575,31.9373,1352.3869,1160.7631
2,2020-01-29 10:42:41,1001001118649,1159.2,sieve-b2w-americanas,1256.575,31.9373,1352.3869,1160.7631
3,2019-11-19 11:23:43,1001001118649,1260.0,olist-mercadolivre,1256.575,31.9373,1352.3869,1160.7631
4,2019-11-19 11:02:48,1001001118649,1260.0,olist-cnova,1256.575,31.9373,1352.3869,1160.7631


In [55]:
comp_m.shape

(45919, 8)

In [56]:
# filter comp_m on extremely low and high prices
comp_m = comp_m[(comp_m['value']>= comp_m['lower']) & (comp_m['value']<= comp_m['upper'])]

In [57]:
comp_m.shape

(45412, 8)

In [58]:
# Group by freq and get minimum value
comp_m = comp_m.groupby([pd.Grouper(key='updated_at', freq=freq),'gtin']).min()[['value']].reset_index()

In [59]:
comp_m.head()

,updated_at,gtin,value
0,2018-08-22,6941057400297,699.90
1,2018-08-22,7897375164269,12.00
2,2018-08-22,7898066740533,120.90
3,2018-08-22,7898605640461,82.38
4,2018-08-22,7899459304660,39.90


##### merge competition prices with merged dataframe

In [ ]:
#merged['day'] = merged['date'].apply(lambda x: x.date())

In [60]:
merged = pd.merge(merged, comp_m,
         left_on=['date','product_gtin'], right_on=['updated_at', 'gtin'],
         how='left')

merged.rename(columns={'value':'competition_price'}, inplace=True)

merged = merged[['date', 'product_gtin',  'competition_price', 'price', 'freight_value',
       'commission_product_olist', 'commission_freight_olist', 'orders']].copy()

In [61]:
merged.head()

,date,product_gtin,competition_price,price,freight_value,commission_product_olist,commission_freight_olist,orders
0,2017-01-06,7891530258930,NaN,579.99,28.55,0.2,0.0,1.0
1,2017-01-08,7898914878487,NaN,66.90,18.75,0.2,0.0,1.0
2,2017-01-08,7898914878487,NaN,66.90,18.75,0.2,0.0,1.0
3,2017-01-09,7898914878487,NaN,66.90,18.75,0.2,0.0,2.0
4,2017-01-09,7898914878487,NaN,66.90,18.75,0.2,0.0,2.0


In [62]:
merged.sort_values(by=['product_gtin', 'date'])

,date,product_gtin,competition_price,price,freight_value,commission_product_olist,commission_freight_olist,orders
19707,2020-02-10,1001001118649,NaN,1260.00,104.470,0.200,0.200,1.0
15532,2019-12-11,1003001113372,NaN,5079.94,263.310,0.200,0.200,1.0
16953,2020-01-06,1003001209488,939.00,939.00,34.620,0.200,0.200,1.0
18046,2020-01-20,1003001209488,NaN,939.00,25.210,0.200,0.200,1.0
9452,2019-08-12,1234567896618,NaN,98.69,22.220,0.190,0.190,1.0
10457,2019-09-08,1234567896618,NaN,98.69,22.220,0.200,0.200,1.0
11022,2019-09-22,1234567896618,NaN,98.69,22.220,0.200,0.200,1.0
11975,2019-10-16,1234567896618,NaN,98.69,26.610,0.200,0.200,1.0
12811,2019-11-04,1234567896618,NaN,98.69,40.360,0.200,0.200,1.0
13196,2019-11-11,1234567896618,NaN,98.69,24.415,0.200,0.200,2.0


We will forward fill the competition price.

In [ ]:
merged['competition_price'].fillna(method='ffill', inplace=True)

In [ ]:
merged.head()

In [ ]:
merged.tail()

### Product type and group

Let's gather some information about the type of the analyzed product.

This particular gtin belongs to the product_type "Kit para cabelo" (Hair Care Kit).  
The product belongs to the broader group "beleza_saude" (beauty_health)

In [ ]:
product_type = pd.read_csv('data/product_type.csv', dtype={'value':str})

product_type.head()

product_type[product_type['value']==gtin]

merged.columns

merged = pd.merge(left=merged, right=product_type,
         left_on='product_gtin', right_on='value')[['date', 'product_gtin', 'competition_price',
                                                    'price', 'freight_value', 'commission_product_olist', 
                                                    'commission_freight_olist', 'orders','product_type', 
                                                    'product_type_group']]

In [ ]:
merged.head()

### Olist history of prices
### TODO

For this particular case, we will only read historical data for the selected gtin. This will be changed once olist can share data on other products.  
Due to internal issues, the price history could not be gathered.


In [ ]:
# history = pd.read_csv('data/product_history.csv', dtype={'gtin':str})
# history['updated_at'] = pd.to_datetime(history['updated_at'])

### SOME CHARTS

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [ ]:
merged.columns

In [ ]:
trace0 = go.Scatter(x=merged.date,
                   y=merged.orders, mode='lines')

trace1 = go.Scatter(x=merged.date,
                    y=merged.competition_price, mode='markers')

trace2 = go.Scatter(x=merged.date,
                    y=merged.price,
                    mode='markers')

trace3 = go.Scatter(x=merged.date,
                    y=merged.freight_value,
                    mode='lines')


iplot([trace0, trace1, trace2, trace3])

In [ ]:
plt.figure(figsize=(12,8))

plt.style.use('fivethirtyeight')

sns.lineplot(x='date', y='orders', data=merged)
sns.scatterplot(x='date', y='competition_price', data=merged, label='competitor')
sns.scatterplot(x='date', y='price', data=merged, label='olist')

plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
sum(merged['orders'])